In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import networkx as nx
from volcapy.grid.grid_from_dsm import Grid
from volcapy.uq.set_estimation import vorobev_expectation_inds


# Load Niklas data.
#data_folder = "/home/cedric/PHD/Dev/VolcapySIAM/data/InversionDatas/stromboli_173018/"
data_folder = "/storage/homefs/ct19x463/Data/InversionDatas/stromboli_173018/"
output_folder = "/storage/homefs/ct19x463/Volcano_DP_results/experiments/cost_unaware_dynamic_strategy/"

data_coords = np.load(os.path.join(data_folder,"niklas_data_coords_corrected_final.npy"))
data_coords_inds_insurf = np.load(os.path.join(data_folder,"niklas_data_inds_insurf.npy"))[1:] # Remove base station.
surface_data_coords = np.load(os.path.join(data_folder,"surface_data_coords.npy"))
G_full_surface = torch.from_numpy(np.load(os.path.join(data_folder,"F_full_surface.npy")))

grid = Grid.load(os.path.join(data_folder, "grid.pickle"))                                             
volcano_coords = torch.from_numpy(grid.cells).float().detach() 

from volcapy.graph.loading import load_Stromboli_graphs
graph_trails_insurf, graph_surface, graph_merged = load_Stromboli_graphs(data_folder) 

# Prepare experiment.
from volcapy.prepare_DP_experiments import prepare_experiment_data
sample_nr = 1
(trained_gp_model, ground_truth, volcano_grid,
     data_coords, G, data_feed,
     accessibility_graph, base_station_node,
     THRESHOLD_small, THRESHOLD_big, _) = prepare_experiment_data(sample_nr)

In [ ]:
# Define the (blind) critetion to optimize.
lower_thresold = torch.Tensor([2600.0]).to('cuda')
data_std = 0.1 

G_full_surface = torch.from_numpy(G)

def compute_blind_wIVR(design, belief):                                                                                                                                                                            
    # Get the observation operator corresponding to the design.                     
    G = G_full_surface[np.array(design), :]                                                                                                                
    current_coverage = belief.coverage(lower_thresold, None)                        
    variance_reduction, _ = belief.covariance.compute_VR(G, data_std)                                             
    wIVR = torch.sum(variance_reduction * current_coverage)
    IVR = torch.sum(variance_reduction)

    return {'reward': wIVR.item(), 'reward_metadata': {'IVR': IVR.item()}}
                                                                                  
def compute_mismatch(ground_truth, estimated_excursion_inds):  
          true_excursion_inds = (ground_truth >= lower_thresold.cpu()).nonzero()[:, 0]       
                                                                                  
          # Compute mismatch.                                                     
          mismatch = np.zeros(ground_truth.shape[0])                            
          mismatch[estimated_excursion_inds] = 1                                        
          tmp = np.zeros(ground_truth.shape[0])                                 
          tmp[true_excursion_inds] = 2                                            
          mismatch = mismatch + tmp                                               
                                                                                  
          excu_size = true_excursion_inds.shape[0] + 1                            
          p_false_pos = 100 * np.sum(mismatch == 1) / excu_size                   
          p_false_neg = 100 * np.sum(mismatch == 2) / excu_size                   
          p_correct = 100 * np.sum(mismatch == 3) / excu_size                     
                                                                                  
          return (p_false_pos, p_false_neg, p_correct)          

def compute_accuracy_metrics(ground_truth, design, belief, already_updated=False):
    if not torch.is_tensor(ground_truth): ground_truth = torch.from_numpy(ground_truth)
    if already_updated is False:
        # Get the data.
        # Get the observation operator corresponding to the design.                     
        G = G_full_surface[np.array(design), :]
        y = data_feed(np.array(design))
    
        # Add the data.
        belief.update(G, y, data_std)
    
    # Compute the new variance and coverage.
    variance_new = belief.covariance.extract_variance().cpu()
    coverage_new = belief.coverage(lower_thresold, None).cpu()
    
    # Residual weighted integrated variance.
    wIV_new = torch.sum(variance_new * coverage_new)
    IBV_final = torch.sum(coverage_new * (1 - coverage_new))
    
    # Error with plug-in estimate.
    post_mean = belief.mean_vec.cpu()
    vorobev_excursion_inds = vorobev_expectation_inds(coverage_new)
    p_false_pos, p_false_neg, p_correct = compute_mismatch(ground_truth, vorobev_excursion_inds)

    if already_updated is False:
        # Return GP to original state.
        belief.rewind(0)
    return {'wIV_final': wIV_new.item(),
            'IBV_final': IBV_final.item(), 'p_false_pos_final': p_false_pos, 'p_false_neg_final': p_false_neg, 'p_correct_final': p_correct}

In [ ]:
belief = trained_gp_model

In [ ]:
from volcapy.strategy.cost_unaware_dynamic_strategies import cost_unaware_wIVR

budget = 60 * 60 * 3 # 3 hours.  


evaluated_designs, rewards, rewards_metadata, costs, paths, accuracy_metrics = [], [], [], [], [], []

# Sample a starting node at random.
from volcapy.set_sampling import uniform_size_uniform_sample
starting_node= uniform_size_uniform_sample(
            list(accessibility_graph.nodes), sample_size=1,
            min_size=1, max_size=1)

best_path, observed_data, best_design = cost_unaware_wIVR(
    belief, budget, accessibility_graph, base_station_node, starting_node[0],
    G, data_feed,
    THRESHOLD_small)

# Arrange results to be saved.
evaluated_designs.append(best_design)
paths.append(best_path)
costs.append(nx.path_weight(accessibility_graph, best_path, weight='weight'))

In [ ]:
# Compute different metrics for comparison.
accuracy_metric = compute_accuracy_metrics(ground_truth, design=best_design, belief=belief, already_updated=True)
accuracy_metrics.append(accuracy_metric)

# Have to reset belief to compute blind reward.
belief.rewind(0)
blind_reward = compute_blind_wIVR(best_design, belief)

rewards.append(blind_reward['reward'])                      
rewards_metadata.append(blind_reward['reward_metadata'])

df = pd.DataFrame.from_dict(
    {'design': evaluated_designs, 'reward': rewards, 'reward_metadata': rewards_metadata,
     'cost': costs, 'path': paths})                                                         
df_accuracy_metric = pd.DataFrame(accuracy_metrics)                                                                           
df = pd.concat([df, df_accuracy_metric], axis=1)

df.to_pickle("results_cost_unaware_wIVR.pkl")

In [ ]:
df

In [ ]:
best_design

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 15))
plt.scatter(data_coords[:, 0], data_coords[:, 1])
plt.scatter(data_coords[best_path, 0], data_coords[best_path, 1], color='yellow')
plt.scatter(data_coords[base_station_node, 0], data_coords[base_station_node, 1], color='red', s=300)
plt.scatter(data_coords[starting_node, 0], data_coords[starting_node, 1], color='green', s=300)
end_node = best_design[-1]
plt.scatter(data_coords[end_node, 0], data_coords[end_node, 1], color='blue', s=300)
plt.savefig("cost_unaware_wIVR_plan.pdf")

In [ ]:
print(accuracy_metrics)

In [ ]:
from volcapy.strategy.static_strategies import static_blind_path_selection
from volcapy.graph.cost_functions import symmetric_walking_cost_fn_edge

from volcapy.set_sampling import uniform_size_uniform_sample

budget = 60 * 60 * 14 # 14 hours.  
n_starting_designs = 2000
N_refine = n_starting_designs # Return all evaluated designs.

# Only sample sets that contain the base node.
base_node = list(graph_trails_insurf.nodes)[0]
sampler = lambda x, y: uniform_size_uniform_sample(x, y, min_size=2, max_size=30, fixed_node=base_node)

output_file = os.path.join(output_folder, "results_sample_{}.pkl".format(sample_nr))
results_df = static_blind_path_selection(
    compute_blind_wIVR, compute_accuracy_metrics,
    belief, budget, N_refine, graph_merged, symmetric_walking_cost_fn_edge, design_sampler=sampler,
    n_starting_designs=n_starting_designs,
    output_path=output_file)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(results_df['cost'] / 60**2)

In [ ]:
filtered_results = results_df[results_df['reward'] > -np.inf]

In [ ]:
wIV_finals = [x[0].item() for x in filtered_results['accuracy_metrics']]
p_correct_finals = [x[-1] for x in filtered_results['accuracy_metrics']]
p_false_pos_finals = [x[1] for x in filtered_results['accuracy_metrics']]
p_false_neg_finals = [x[2] for x in filtered_results['accuracy_metrics']]

In [ ]:
plt.subplot(131)
plt.scatter(list(filtered_results['reward']), p_correct_finals)
plt.subplot(132)
plt.scatter(list(filtered_results['reward']), p_false_pos_finals)
plt.subplot(133)
plt.scatter(list(filtered_results['reward']), p_false_neg_finals)

In [ ]:
G_tmp = G_full_surface[np.array(best_designs[1]), :]
current_coverage = belief.coverage(lower_thresold, None)    
wIVR = belief.IVR(G_tmp, data_std,                                                  
            weights=current_coverage)

In [ ]:
base_node = list(graph_trails_insurf.nodes)[0]
graph_merged.edges(base_node, data=True)